In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark pandas numpy

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.2.2-bin-hadoop3.2')


from pyspark.sql import functions as func
from pyspark.sql.functions import sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, concat
from pyspark.sql.functions import expr
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import warnings, re

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


# Instanciando Spark
spark = SparkSession.builder.master('local[*]').getOrCreate()

--2023-09-03 22:01:31--  https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz’

spark-3.2.2-bin-had 100%[===================>] 287.16M  23.0MB/s    in 13s     

2023-09-03 22:01:44 (21.7 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz’ saved [301112604/301112604]



In [3]:
spark = SparkSession.builder.appName("example").getOrCreate()


In [4]:
# 01- Como criar um Dataframe
columns = "Name string, Age int"
data = [("Alice", 28), ("Bob", 22), ("Charlie", 35), ("Charlie", 35),("Charlie", 35),("Charlie", 35),("Charlie", 35)]
df = spark.createDataFrame(data, columns)
df.show()


+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 28|
|    Bob| 22|
|Charlie| 35|
|Charlie| 35|
|Charlie| 35|
|Charlie| 35|
|Charlie| 35|
+-------+---+



In [5]:
# 02- Como criar um Dataframe simples, sem um schema
from pyspark.sql import SparkSession
df1 = spark.createDataFrame([("Pedro",10),("Maria",20),("José",40)])

In [6]:
df1.show()

+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| José| 40|
+-----+---+



In [7]:
schema = "ID INT, NOME STRING"
dados = [[1,"PEDRO"],[2,"MARIA"]]
df2=spark.createDataFrame(dados,schema)
df2.show()

+---+-----+
| ID| NOME|
+---+-----+
|  1|PEDRO|
|  2|MARIA|
+---+-----+



In [8]:
df2.show(1)

+---+-----+
| ID| NOME|
+---+-----+
|  1|PEDRO|
+---+-----+
only showing top 1 row



In [9]:
#03-Criando um Datframe  simples com Schema
schema2 = "Produtos STRING, Vendas INT"
vendas = [["Caneta",10],["Lápis",20],["Caneta",40]]
df3 = spark.createDataFrame(vendas , schema2 )
df3.show()


+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [10]:
schema2 = "Produtos STRING, Vendas INT"
vendas = [["Caneta",10],["Lápis",20],["Caneta",40]]
df3 = spark.createDataFrame(vendas , schema2 )
df3.show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [11]:
schema = "Produto string, Quantidade int"
dados = [["carne",100],["biscoito",100]]
df5 = spark.createDataFrame(dados, schema)
df5.show()

+--------+----------+
| Produto|Quantidade|
+--------+----------+
|   carne|       100|
|biscoito|       100|
+--------+----------+



In [12]:
#04- Criando um Dataframe com uma structype.


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("ExemploSpark").getOrCreate()

schema = StructType([
    StructField("matricula", IntegerType(), True),
    StructField("Nome", StringType(), True),
    StructField("área", StringType(), True),
    StructField("tad", StringType(), True)
])

colunas = "matricula int, Nome string, área string, tad string"
dados = [
    [40213, "marcus", "logística", "Recebimento"],
    [40214, "joão", "produção", "Produção"],
    [40215, "ana", "vendas", "Vendas"]
]

df7 = spark.createDataFrame(dados, schema)
df7.show()


+---------+------+---------+-----------+
|matricula|  Nome|     área|        tad|
+---------+------+---------+-----------+
|    40213|marcus|logística|Recebimento|
|    40214|  joão| produção|   Produção|
|    40215|   ana|   vendas|     Vendas|
+---------+------+---------+-----------+



In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import sum

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("ExemploSpark").getOrCreate()

colunas = StructType([
    StructField("Produto", StringType(), True),
    StructField("Vendas", IntegerType(), True)])

linhas = [
["Caneta", 10000],
["Caneta", 5000],
["Resma", 80000],
["Lápis", 2000],
["Caderno", 10000],
["Mochila", 10000],
]
dfmarcus = spark.createDataFrame(linhas, colunas)
dfmarcus.show()

+-------+------+
|Produto|Vendas|
+-------+------+
| Caneta| 10000|
| Caneta|  5000|
|  Resma| 80000|
|  Lápis|  2000|
|Caderno| 10000|
|Mochila| 10000|
+-------+------+



In [14]:
dfmarcus.show(2)

+-------+------+
|Produto|Vendas|
+-------+------+
| Caneta| 10000|
| Caneta|  5000|
+-------+------+
only showing top 2 rows



In [15]:
#05 Como usar a função de agregação com groupby
# Primeiro nos agrupamos de depois aplicamos a função de agregação

dfsoma = dfmarcus.groupBy("Produto").agg(sum("Vendas"))
dfsoma.show()

+-------+-----------+
|Produto|sum(Vendas)|
+-------+-----------+
| Caneta|      15000|
|  Resma|      80000|
|Caderno|      10000|
|  Lápis|       2000|
|Mochila|      10000|
+-------+-----------+



In [16]:
#06- Criando um Datframe

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import sum

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("ExemploSpark").getOrCreate()

colunas = StructType([
    StructField("vendedor", StringType(), True),
    StructField("Produto", StringType(), True),
    StructField("Região", StringType(), True),
    StructField("Estado", StringType(), True),
    StructField("Valor de Vendas", IntegerType(), True)
    ])

linhas = [
["Ricardo", "KiaSportage", "Nordeste", "Bahia", 10000],
["Paulo", "Vectra", "Parana", "Bahia", 10000],
["Carlos", "KiaSportage", "Nordeste", "Bahia", 10000],
["Ricardo", "KiaSportage", "Nordeste", "Bahia", 10000],
["Ricardo", "KiaSportage", "Nordeste", "Bahia", 10000],
]
dfmarcus = spark.createDataFrame(linhas, colunas)
dfmarcus.show()

+--------+-----------+--------+------+---------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|
+--------+-----------+--------+------+---------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
|   Paulo|     Vectra|  Parana| Bahia|          10000|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
+--------+-----------+--------+------+---------------+



In [17]:
#07- Como colocar um apelido para uma coluna que sofre ação da função de agrageção
dfsoma = dfmarcus.groupBy("vendedor").agg(sum("Valor de Vendas").alias("total de vendas"))
dfsoma.show()

+--------+---------------+
|vendedor|total de vendas|
+--------+---------------+
|   Paulo|          10000|
| Ricardo|          30000|
|  Carlos|          10000|
+--------+---------------+



In [18]:
#08- Como usar o comando select em um dataframe para vermos as colunas que queremos
dfmarcus.show()

+--------+-----------+--------+------+---------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|
+--------+-----------+--------+------+---------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
|   Paulo|     Vectra|  Parana| Bahia|          10000|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
+--------+-----------+--------+------+---------------+



In [19]:
#09- Como concatenar duas colunas que no nosso caso são, vendedor e produto

from pyspark.sql.functions import col, concat
dfmarcus2 = dfmarcus.withColumn("concateda", concat(col("vendedor"), col("produto")))
dfmarcus2.show()



+--------+-----------+--------+------+---------------+------------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|         concateda|
+--------+-----------+--------+------+---------------+------------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|RicardoKiaSportage|
|   Paulo|     Vectra|  Parana| Bahia|          10000|       PauloVectra|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000| CarlosKiaSportage|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|RicardoKiaSportage|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|RicardoKiaSportage|
+--------+-----------+--------+------+---------------+------------------+



In [20]:

#10- Como concatenar letras de nomes em colunas diferentes, no nosso exemplo foi
# as tres primeiras letras do nomes da coluna vendedor, com as três ultimas letras
# do nome produto

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, substring

dfmarcus3 = dfmarcus.withColumn("Nome_concatenado", concat(substring(col("vendedor"), 1, 3), substring(col("produto"), -3, 3)))
dfmarcus3.show()


+--------+-----------+--------+------+---------------+----------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|Nome_concatenado|
+--------+-----------+--------+------+---------------+----------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|          Ricage|
|   Paulo|     Vectra|  Parana| Bahia|          10000|          Pautra|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000|          Carage|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|          Ricage|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|          Ricage|
+--------+-----------+--------+------+---------------+----------------+



In [21]:
dfmarcus.show()

+--------+-----------+--------+------+---------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|
+--------+-----------+--------+------+---------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
|   Paulo|     Vectra|  Parana| Bahia|          10000|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|
+--------+-----------+--------+------+---------------+



In [22]:
#11- Como multiplicar o valor de uma coluna por detrminado fator ou seja valor.

dfmarcus3= dfmarcus.withColumn("Quantidade_Duplicada", col("valor de vendas") * 5)
dfmarcus3.show()

+--------+-----------+--------+------+---------------+--------------------+
|vendedor|    Produto|  Região|Estado|Valor de Vendas|Quantidade_Duplicada|
+--------+-----------+--------+------+---------------+--------------------+
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|               50000|
|   Paulo|     Vectra|  Parana| Bahia|          10000|               50000|
|  Carlos|KiaSportage|Nordeste| Bahia|          10000|               50000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|               50000|
| Ricardo|KiaSportage|Nordeste| Bahia|          10000|               50000|
+--------+-----------+--------+------+---------------+--------------------+



In [23]:
#12- Como usar o expr.

from pyspark.sql.functions import expr

dfmarcus4 = dfmarcus.select("Produto", "Valor de Vendas",
expr("`Valor de Vendas` * 5").alias("Valor_Vendas_Multiplicado"))
dfmarcus4.show()



+-----------+---------------+-------------------------+
|    Produto|Valor de Vendas|Valor_Vendas_Multiplicado|
+-----------+---------------+-------------------------+
|KiaSportage|          10000|                    50000|
|     Vectra|          10000|                    50000|
|KiaSportage|          10000|                    50000|
|KiaSportage|          10000|                    50000|
|KiaSportage|          10000|                    50000|
+-----------+---------------+-------------------------+



In [24]:
#13- Como verificar o schema do Dataframe.
dfmarcus4.schema

StructType(List(StructField(Produto,StringType,true),StructField(Valor de Vendas,IntegerType,true),StructField(Valor_Vendas_Multiplicado,IntegerType,true)))

In [25]:
#14- Verificar as colunas de um dataframe.

dfmarcus4.columns

['Produto', 'Valor de Vendas', 'Valor_Vendas_Multiplicado']

In [27]:
#15- Como importamos um arquivo para dentro de um data frame.
#Nós determinando os nomes das colunas ou seja o schema.


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import sum

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("ExemploSpark").getOrCreate()

colunas = StructType([
    StructField("ID", IntegerType(), True),
    StructField("NOME", StringType(), True),
    StructField("STATUS", StringType(), True),
    StructField("CIDADE", StringType(), True),
    StructField("VENDAS", IntegerType(), True),
    StructField("DATA", StringType(), True)
    ])

dfdespachantes = spark.read.csv("/home/pyspark/despachantes.csv",
header=False, schema=colunas)

dfdespachantes.show()




+---+-------------------+------+-------------+------+----------+
| ID|               NOME|STATUS|       CIDADE|VENDAS|      DATA|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [28]:
#16- Como importamos um arquivo para dentro de um data frame.
#Pedindo ao spark para ele inferir o schema.

dfautoschema = spark.read.csv("/home/pyspark/despachantes.csv",
sep=",", inferSchema=True, header=False)
dfautoschema.show()


+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [29]:
#17- Como usar a cláusula Where no Pyspark, Qual a Sintaxe?
dfdespachantes .select ("id", "nome", "vendas") \
.where ((dfdespachantes["vendas"] > 20)).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+



In [30]:
#18- Usando o operador Logico & (AND)

dfdespachantes .select ("id", "nome", "vendas") \
.where((col("vendas") > 20)\
& (col("vendas") < 30)).show()



+---+----------------+------+
| id|            nome|vendas|
+---+----------------+------+
|  1|Carminda Pestana|    23|
|  6|Matilde Rebouças|    22|
+---+----------------+------+



In [31]:
#19- Usando o operador Logico | (OR)

df_filtered = dfdespachantes.select("id", "nome", "vendas") \
    .where((col("vendas") > 20) & (col("vendas") < 30)) \
    .show()

+---+----------------+------+
| id|            nome|vendas|
+---+----------------+------+
|  1|Carminda Pestana|    23|
|  6|Matilde Rebouças|    22|
+---+----------------+------+



In [32]:
#20- Usando o operador Logico != (Diferente)

dfdespachantes.select("id", "nome", "vendas") \
    .where((col("vendas") != 1)) \
    .show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  5|     Graça Ornellas|    12|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
| 10|   Viviana Sequeira|     0|
+---+-------------------+------+



In [33]:
#21- Usando o operador Logico - (nega a afirmação)
dfdespachantes.select("id", "nome", "vendas") \
    .where(~ (col("vendas") == 1)) \
    .show()


+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  5|     Graça Ornellas|    12|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
| 10|   Viviana Sequeira|     0|
+---+-------------------+------+



In [34]:
#22- Usando o operador Logico == (Igual)
dfdespachantes.select("id", "nome", "vendas")\
.where((col("vendas") == 23)).show()

+---+----------------+------+
| id|            nome|vendas|
+---+----------------+------+
|  1|Carminda Pestana|    23|
+---+----------------+------+



In [35]:
#23- Como renomear uma coluna.
dfnovo=dfdespachantes.withColumnRenamed("ID", "IDENTIFICAÇÃO")
dfnovo.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|               NOME|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [36]:
#24- Como renomear várias colunas ao mesmo tempo.
dfnovo=dfdespachantes.withColumnRenamed("ID", "IDENTIFICAÇÃO")\
.withColumnRenamed("NOME", "PESSOAS")\
.withColumnRenamed("VENDAS", "TOTAL DE VENDAS")
dfnovo.show()

+-------------+-------------------+------+-------------+---------------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|TOTAL DE VENDAS|      DATA|
+-------------+-------------------+------+-------------+---------------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|             23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|             34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|             34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|             36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|             12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|             22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|             45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|             65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|             54|2018

In [37]:
#25- Como renomear colunas e criar um novo dataframe.
dfnovo1=dfnovo.withColumnRenamed("TOTAL DE VENDAS", "VENDAS")
dfnovo1.show()


+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [38]:
#26- Como usar a função de agregação SUM em uma coluna.
from pyspark.sql.functions import count, desc

dfnovo1.groupBy("PESSOAS").agg(count(col("VENDAS").alias('VENDAS')))\
.orderBy(desc(col("PESSOAS"))).show()




+-------------------+-----------------------+
|            PESSOAS|count(VENDAS AS VENDAS)|
+-------------------+-----------------------+
|   Viviana Sequeira|                      1|
|      Uriel Queiroz|                      1|
|      Roque Vásquez|                      1|
|    Noêmia   Orriça|                      1|
|   Matilde Rebouças|                      1|
|     Graça Ornellas|                      1|
|Felisbela Dornelles|                      1|
|   Emídio Dornelles|                      1|
|    Deolinda Vilela|                      1|
|   Carminda Pestana|                      1|
+-------------------+-----------------------+



In [39]:
#27- Como selecionar todas as colunas do DataFrame dfnovo1.
# Aplicar a operação distinct.
# Remover as linhas duplicadas.
# O método show() é usado para exibir o resultado na saída.

dfnovo1.select("*").distinct()
dfnovo1.show()


+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [40]:
dfnovo1.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [41]:
#28- Realizando groupBY por uma coluna e criando um data frame novo.
# Forma01

df03 = dfnovo1.groupBy("status").agg(sum(col("vendas")))
df03.show()

+------+-----------+
|status|sum(vendas)|
+------+-----------+
| Ativo|        325|
+------+-----------+



In [42]:
#29- Realizando groupBY por uma coluna e criando um data frame novo.
# Forma 02

df03 = dfnovo1.groupBy("status").agg(sum(col("vendas"))).show()

+------+-----------+
|status|sum(vendas)|
+------+-----------+
| Ativo|        325|
+------+-----------+



In [43]:
#20- Realizando o agrupamento por múltiplas colunas.
# Criando um data frame novo.
# Forma 01
df03 = dfnovo1.groupBy("status","pessoas").agg(sum(col("vendas")))
df03.show()

+------+-------------------+-----------+
|status|            pessoas|sum(vendas)|
+------+-------------------+-----------+
| Ativo|      Uriel Queiroz|         54|
| Ativo|      Roque Vásquez|         65|
| Ativo|   Emídio Dornelles|         34|
| Ativo|    Deolinda Vilela|         34|
| Ativo|   Carminda Pestana|         23|
| Ativo|    Noêmia   Orriça|         45|
| Ativo|     Graça Ornellas|         12|
| Ativo|   Matilde Rebouças|         22|
| Ativo|Felisbela Dornelles|         36|
| Ativo|   Viviana Sequeira|          0|
+------+-------------------+-----------+



In [44]:
#31- Realizando o agrupamento por múltiplas colunas.
# Criando um data frame novo.
# Forma 02
df03 = dfnovo1.groupBy("status","pessoas").agg(sum(col("vendas"))).show()

+------+-------------------+-----------+
|status|            pessoas|sum(vendas)|
+------+-------------------+-----------+
| Ativo|      Uriel Queiroz|         54|
| Ativo|      Roque Vásquez|         65|
| Ativo|   Emídio Dornelles|         34|
| Ativo|    Deolinda Vilela|         34|
| Ativo|   Carminda Pestana|         23|
| Ativo|    Noêmia   Orriça|         45|
| Ativo|     Graça Ornellas|         12|
| Ativo|   Matilde Rebouças|         22|
| Ativo|Felisbela Dornelles|         36|
| Ativo|   Viviana Sequeira|          0|
+------+-------------------+-----------+



In [45]:
#32- Realizando groupBY por uma coluna sem criar um data frame novo.
# Forma01

dfnovo1.groupBy("status").agg(sum(col("vendas"))).show()

+------+-----------+
|status|sum(vendas)|
+------+-----------+
| Ativo|        325|
+------+-----------+



In [46]:
#33- Realizando groupBY por uma coluna sem criar um data frame novo.
# Forma02

dfnovo1.groupBy(col("PESSOAS")).agg(sum(col("vendas"))).show()

+-------------------+-----------+
|            PESSOAS|sum(vendas)|
+-------------------+-----------+
|   Matilde Rebouças|         22|
|      Roque Vásquez|         65|
|Felisbela Dornelles|         36|
|    Noêmia   Orriça|         45|
|   Emídio Dornelles|         34|
|    Deolinda Vilela|         34|
|   Viviana Sequeira|          0|
|      Uriel Queiroz|         54|
|   Carminda Pestana|         23|
|     Graça Ornellas|         12|
+-------------------+-----------+



In [47]:
#34- Como trabalhamos com groupby e orderBY e desc e Asc.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
dfnovo1.groupBy("PESSOAS").agg(sum(col("VENDAS"))).orderBy(desc(col("PESSOAS"))).show()


+-------------------+-----------+
|            PESSOAS|sum(VENDAS)|
+-------------------+-----------+
|   Viviana Sequeira|          0|
|      Uriel Queiroz|         54|
|      Roque Vásquez|         65|
|    Noêmia   Orriça|         45|
|   Matilde Rebouças|         22|
|     Graça Ornellas|         12|
|Felisbela Dornelles|         36|
|   Emídio Dornelles|         34|
|    Deolinda Vilela|         34|
|   Carminda Pestana|         23|
+-------------------+-----------+



In [48]:
dfnovo1.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [49]:
#35- Como usar o Distinct no pyspark.
dfdistinct = dfnovo1.select("CIDADE","VENDAS").distinct()
dfdistinct.show()

+-------------+------+
|       CIDADE|VENDAS|
+-------------+------+
|  Santa Maria|    45|
| Porto Alegre|    34|
|  Santa Maria|    23|
|Novo Hamburgo|    34|
| Porto Alegre|    65|
| Porto Alegre|    22|
| Porto Alegre|    36|
| Porto Alegre|    12|
| Porto Alegre|    54|
| Porto Alegre|     0|
+-------------+------+



In [50]:
#36- Como usar o dropDuplicates.

dfdistinct2=dfdistinct.select("VENDAS").dropDuplicates()
dfdistinct2.show()


+------+
|VENDAS|
+------+
|    65|
|    34|
|    12|
|    22|
|    54|
|    23|
|    45|
|     0|
|    36|
+------+



In [51]:
#37- Como usar o comando filter.
dfnovo1.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [52]:
#38- Como usar o comando filter
from pyspark.sql.functions import col

dfnovo1.filter(col("PESSOAS") == "Uriel Queiroz").show()

+-------------+-------------+------+------------+------+----------+
|IDENTIFICAÇÃO|      PESSOAS|STATUS|      CIDADE|VENDAS|      DATA|
+-------------+-------------+------+------------+------+----------+
|            9|Uriel Queiroz| Ativo|Porto Alegre|    54|2018-05-05|
+-------------+-------------+------+------------+------+----------+



In [53]:
#39-Como exportar (salvar) dataframes em diversos formatos
# formarto parquet

dfnovo1.write.format("parquet").save("/home/pyspark/Exportando_Dados/dfnovo1aparquet")

In [54]:
#40- Como exportar (salvar) dataframes em diversos formatos
# formarto csv

dfnovo1.write.format("csv").save("/home/pyspark/Exportando_Dados/dfnovo1csv")

In [55]:
#41- Como exportar (salvar) dataframes em diversos formatos
# formarto json

dfnovo1.write.format("json").save("/home/pyspark/Exportando_Dados/dfnovo1json")

In [56]:
#42- Como exportar (salvar) dataframes em diversos formatos
# formarto json

dfnovo1.write.format("orc").save("/home/pyspark/Exportando_Dados/dfnovo1orc")

In [63]:
#43- Como ler os dataframes em diversos formatos
# formarto parquet

dfleitura = spark.read.format("parquet").load("/home/pyspark/Exportando_Dados/dfnovo1aparquet/despachantes.parquet")
dfleitura.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [60]:
#44- Como ler os dataframes em diversos formatos
# formarto json

dfleitura2 = spark.read.format("json").load("/home/pyspark/Exportando_Dados/dfnovo1json/despachantes.json")
dfleitura2.show()

+-------------+----------+-------------+-------------------+------+------+
|       CIDADE|      DATA|IDENTIFICAÇÃO|            PESSOAS|STATUS|VENDAS|
+-------------+----------+-------------+-------------------+------+------+
|  Santa Maria|2020-08-11|            1|   Carminda Pestana| Ativo|    23|
|Novo Hamburgo|2020-03-05|            2|    Deolinda Vilela| Ativo|    34|
| Porto Alegre|2020-02-05|            3|   Emídio Dornelles| Ativo|    34|
| Porto Alegre|2020-02-05|            4|Felisbela Dornelles| Ativo|    36|
| Porto Alegre|2020-02-05|            5|     Graça Ornellas| Ativo|    12|
| Porto Alegre|2019-01-05|            6|   Matilde Rebouças| Ativo|    22|
|  Santa Maria|2019-10-05|            7|    Noêmia   Orriça| Ativo|    45|
| Porto Alegre|2020-03-05|            8|      Roque Vásquez| Ativo|    65|
| Porto Alegre|2018-05-05|            9|      Uriel Queiroz| Ativo|    54|
| Porto Alegre|2020-09-05|           10|   Viviana Sequeira| Ativo|     0|
+-------------+----------

In [61]:
#45- Como ler os dataframes em diversos formatos
#Formarto json

dfleitura3 = spark.read.format("orc").load("/home/pyspark/Exportando_Dados/dfnovo1orc/despachantes.orc")
dfleitura3.show()

+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------

In [62]:
#46-como ler os dataframes em diversos formatos
#Formarto csv

dfleitura1 = spark.read.format("csv").load("/home/pyspark/Exportando_Dados/dfnovo1csv/despachantes.csv")
dfleitura1.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [ ]:
#47- Colocando a os nomes da coluna no dfleitura01

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import sum

# Inicialize a sessão Spark
spark = SparkSession.builder.appName("ExemploSpark").getOrCreate()

colunas = StructType([
    StructField("IDENTIFICAÇÃO", IntegerType(), True),
    StructField("PESSOAS", StringType(), True),
    StructField("STATUS", StringType(), True),
    StructField("CIDADE", StringType(), True),
    StructField("VENDAS", IntegerType(), True),
    StructField("DATA", StringType(), True)
    ])

df_com_schema = spark.read.format("csv")\
.load("/home/pyspark/Exportando_Dados/dfnovo1csv/despachantes.csv",
schema=colunas)

df_com_schema.show()



+-------------+-------------------+------+-------------+------+----------+
|IDENTIFICAÇÃO|            PESSOAS|STATUS|       CIDADE|VENDAS|      DATA|
+-------------+-------------------+------+-------------+------+----------+
|            1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|            2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|            3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|            4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|            5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|            6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|            7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|            8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|            9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|           10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+-------------+----------